In [4]:
from leecarter import leecarter
import pandas as pd

import numpy as np
import numpy.matlib
from matplotlib import pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

In [2]:
filename = 'death_rates2.txt'

mortality = pd.read_csv(filename, delim_whitespace=True)
mortality = mortality[mortality['Age'] != '110+']
mortality['Age'] = mortality['Age'].astype(int)
mortality['Year'] = mortality['Year'].astype(int)
mortality = mortality[mortality['Age'] <= 100]
mortality['Female_l'] = np.log(mortality['Female'].astype(float))
mortality['Male_l'] = np.log(mortality['Male'].astype(float))
mortality['Total_l'] = np.log(mortality['Total'].astype(float))
mortality = mortality[['Year', 'Age', 'Female_l', 'Male_l', 'Total_l']]
mortality

,Year,Age,Female_l,Male_l,Total_l
0,1958,0,-2.723920,-2.480397,-2.591788
1,1958,1,-5.352562,-5.291540,-5.320972
2,1958,2,-6.357324,-6.197751,-6.272237
3,1958,3,-6.732962,-6.527266,-6.622576
4,1958,4,-7.066751,-6.837297,-6.943382
...,...,...,...,...,...
6867,2019,96,-1.240243,-1.143835,-1.220129
6868,2019,97,-1.123130,-1.046502,-1.108135
6869,2019,98,-1.127061,-0.963167,-1.093425
6870,2019,99,-0.980088,-0.864780,-0.957410


In [5]:
# split train test
def fit_arima(matrix):
    d = []
    for a in range(5):
        for b in range(5):
            for c in range(5):
                mod = ARIMA(matrix, order=(a,b,c))
                res = mod.fit()
                d.append([res.llf, a, b, c])
    return max(d)


def predict(matrix, column: str, age):
    fm = matrix.pivot(index="Age", columns="Year", values=column)
    fm_train = fm.iloc[:,0:round(np.shape(fm)[1]*2/3)]
    # fm_cross_val = fm.iloc[:,round(np.shape(fm)[1]*2/3):]
    fm = fm_train
    fm = fm.to_numpy()
    a_x = fm.mean(axis=1)
    N = np.shape(fm)[0]
    T = np.shape(fm)[1]
    z_xt = fm - np.matlib.repmat(a_x, T, 1).T
    U, S, V = np.linalg.svd(z_xt, full_matrices=True)
    bxkt = S[0] * np.dot(U[:, 0].reshape(N, 1), V[0, :].reshape(T, 1).T)
    eps = z_xt - bxkt

    logm_xt_lcfitted = bxkt + a_x.reshape(N, 1)

    b_x = U[:, 0]/U[:, 0].sum()
    k_t = V[0, :]*S[0]*U[:, 0].sum()
    a_x = a_x + k_t.sum()*b_x
    k_t = k_t - k_t.sum()

    _, par1, par2, par3 = fit_arima(k_t)

    mod = ARIMA(k_t, order=(par1,par2,par3))
    res = mod.fit()
    print(res.summary())

    pred = res.forecast(steps=20)

    return np.exp(a_x[age-1] + b_x[age-1]*pred)

In [8]:
age = 65
column = 'Female_l'

In [6]:
vals = predict(mortality, column, age)

C:\Users\piotr\projekty\diffusion_processes_on_complex_networks\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\piotr\projekty\diffusion_processes_on_complex_networks\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\piotr\projekty\diffusion_processes_on_complex_networks\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\piotr\projekty\diffusion_processes_on_complex_networks\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive pa

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   41
Model:                 ARIMA(4, 2, 3)   Log Likelihood                 -85.583
Date:                Fri, 10 Jun 2022   AIC                            187.167
Time:                        13:40:27   BIC                            200.475
Sample:                             0   HQIC                           191.942
                                 - 41                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0124      0.373     -0.033      0.973      -0.743       0.718
ar.L2         -0.0906      0.213     -0.426      0.670      -0.508       0.327
ar.L3         -0.5726      0.120     -4.768      0.0

C:\Users\piotr\projekty\diffusion_processes_on_complex_networks\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [7]:
mortality[morta]

array([0.0129992 , 0.01277259, 0.01261538, 0.01251007, 0.01242813,
       0.01229361, 0.01209861, 0.01187504, 0.01167919, 0.01154856,
       0.01146434, 0.01137721, 0.01124022, 0.0110499 , 0.01084763,
       0.01068315, 0.01057666, 0.01050296, 0.01041292, 0.01027289])